Signal- und Systemtheorie / Signals and Systems \#24015

Dr. Frank Schultz, Prof. Sascha Spors

Institut für Nachrichtentechnik (INT),
Fakultät für Informatik und Elektrotechnik (IEF),
Universität Rostock

Institute of Communications Engineering,
Faculty of Computer Science and Electrical Engineering,
University of Rostock

**Übung / Exercise 6**, Sommer / Summer 2020

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal

basex = 10  # log frequency axis, either 10 for log10 or 2 for log2

w = np.logspace(-3, 3, num=2**6, base=basex)
f = w / (2*np.pi)
#fs = 1e4 * 2*np.pi  # analog and bilinear are very close, group delay matches
fs = 5e1 * 2*np.pi  # we use this for numerical eval of group delay
fs = 300  # use a nicer number in Hz

figw, figh = 8, 8*10/16

In [ ]:
# Max-Phase System
sz = +2
sp = -1/2
H0 = 2
sys = signal.lti(sz, sp, H0)
w, _, Hphase_max = signal.bode(sys,  w)
gdmax = 10*(1+w**2)/(4*w**4+17*w**2+4)

z, p, k = signal.bilinear_zpk(sz, sp, H0, fs)
_, gdmax_z = signal.group_delay(signal.zpk2tf(z, p, k), w=f, fs=fs)
print(np.allclose(gdmax_z/fs, gdmax))

# Min-Phase System
sz = -2
sp = -1/2
H0 = 2
sys = signal.lti(sz, sp, H0)
w, _, Hphase_min = signal.bode(sys,  w)
gdmin = -6*(w**2-1)/(4*w**4+17*w**2+4)

z, p, k = signal.bilinear_zpk(sz, sp, H0, fs)
_, gdmin_z = signal.group_delay(signal.zpk2tf(z, p, k), w=f, fs=fs)
print(np.allclose(gdmin_z/fs, gdmin))

# Allpass
sz = +2
sp = -2
H0 = 1
sys = signal.lti(sz, sp, H0)
w, _, Hphase_all = signal.bode(sys,  w)
gdall = 4/(w**2+4)

z, p, k = signal.bilinear_zpk(sz, sp, H0, fs)
_, gdall_z = signal.group_delay(signal.zpk2tf(z, p, k), w=f, fs=fs)
print(np.allclose(gdall_z/fs, gdall))

plt.figure(figsize=(figw, figh))
plt.subplot(2, 1, 1)
plt.semilogx(w, Hphase_max, lw=4, basex=basex,
             label=r'$H(s)_\mathrm{max}=2\,\frac{s-2}{s+1/2}$')
plt.semilogx(w, Hphase_min, basex=basex,
             label=r'$H(s)_\mathrm{min}=2\,\frac{s+2}{s+1/2}$')
plt.semilogx(w, Hphase_all, basex=basex,
             label=r'$H(s)_\mathrm{all}=\frac{s-2}{s+2}$')
plt.semilogx(w, Hphase_min+Hphase_all, basex=basex,
             label=r'$H(s)_\mathrm{min}\cdot H(s)_\mathrm{all}$')
#plt.xlabel(r'$\omega$ / (rad/s)')
plt.ylabel(r'$\angle H(\mathrm{j}\omega)$')
plt.title(r'Phase')
plt.xlim(w[0], w[-1])
plt.yticks(np.arange(-45, 180+45, 45))
plt.ylim(-45, 180)
plt.legend()
plt.grid(True, which='both')

plt.subplot(2, 1, 2)
plt.semilogx(w, gdmax, lw=4, basex=basex,
             label=r'$H(s)_\mathrm{max}=2\,\frac{s-2}{s+1/2}$')
plt.semilogx(w, gdmin, basex=basex,
             label=r'$H(s)_\mathrm{min}=2\,\frac{s+2}{s+1/2}$')
plt.semilogx(w, gdall, basex=basex,
             label=r'$H(s)_\mathrm{all}=\frac{s-2}{s+2}$')
plt.semilogx(w, gdmin+gdall, basex=basex,
             label=r'$H(s)_\mathrm{min}\cdot H(s)_\mathrm{all}$')
plt.xlabel(r'$\omega$ / (rad/s)')
plt.ylabel(
    r'$\frac{-\mathrm{d}\,\angle H(\mathrm{j}\omega)}{\mathrm{d}\,\omega}$')
plt.title(r'Group Delay')
plt.xlim(w[0], w[-1])
plt.yticks(np.arange(-0.5, 2.5+0.5, 0.5))
plt.ylim(-0.5, 2.5)
plt.legend()
plt.grid(True, which='both')
plt.savefig('group_delay_AB91F8317C.pdf')

# Check Phase Delay and Group Delay with Bilinear Transformed Filters

In [ ]:
w0 = 1e-1
f0 = w0 / (2*np.pi)

tend = 1200
N = fs*tend+1
n = np.arange(0,N)
t = n / fs
a = 3/800 # 0.00375
print(a)
gausst = np.exp(-a**2 * (t-tend/2)**2)
gaussw = np.sqrt(np.pi/a**2) * np.exp(-w**2/(4*a**2))  # phase not considered
x = np.cos(w0*(t-tend/2)) * gausst

plt.subplot(2,1,1)
plt.plot(w,20*np.log10(gaussw)-20*np.log10(np.max(gaussw)))
plt.xlim(0,w0)
plt.xticks(np.arange(0,w0,0.025))
plt.ylim(-100,0)
plt.xlabel(r'$\omega$ / (rad/s)')
plt.grid(True)

plt.subplot(2,1,2)
plt.plot(t,gausst)
plt.xlabel(r'$t$ / s')
plt.grid(True)

In [ ]:
plt.figure(figsize=(9,12))

# Max-Phase System
sz = +2
sp = -1/2
H0 = 2
sys = signal.lti(sz, sp, H0)
_, Hlvl_db_w0, _ = signal.bode(sys,  w0)
z, p, k = signal.bilinear_zpk(sz, sp, H0, fs)
_, gdmax_z = signal.group_delay(signal.zpk2tf(z, p, k), w=f, fs=fs)
sos = signal.zpk2sos(z, p, k)
gd_w0 = 10*(1+w0**2)/(4*w0**4+17*w0**2+4)
y = signal.sosfilt(sos, x) * 10**(-Hlvl_db_w0/20)
y_env_max_t = np.argmax(np.abs(signal.hilbert(y)))/fs
x_env_max_t = np.argmax(np.abs(signal.hilbert(x)))/fs

plt.subplot(3,1,1)
plt.plot(t,x, 'C0', label=r'input signal $x(t)$')
plt.plot(t,y, 'C1', label=r'output sigal $y(t)$ normalized by system gain')
plt.plot(t,np.abs(signal.hilbert(x)),'C0--', label=r'envelope of $x(t)$')
plt.plot(t,np.abs(signal.hilbert(y)),'C1:', label=r'envelope of $y(t)$')
plt.text(10,0.92, r'analytic result $\tau_\mathrm{GD}$ = %4.3f s' % gd_w0)
plt.text(10,0.82,r'numeric evaluation $\tau_\mathrm{GD}$ = %4.3f s' % (y_env_max_t-x_env_max_t))
plt.xlim(t[0],t[-1])
#plt.xlabel(r'$t$ / s')
plt.ylabel(r'$x(t), y(t)$')
plt.title(r'Maximum Phase System $H(s)_\mathrm{max}=2\,\frac{s-2}{s+1/2}$, $\omega_0=0.1$ rad/s, $\alpha=0.00375$')
plt.legend(loc='upper right')
plt.grid(True)

# Min-Phase System
sz = -2
sp = -1/2
H0 = 2
sys = signal.lti(sz, sp, H0)
_, Hlvl_db_w0, _ = signal.bode(sys,  w0)
z, p, k = signal.bilinear_zpk(sz, sp, H0, fs)
_, gdmax_z = signal.group_delay(signal.zpk2tf(z, p, k), w=f, fs=fs)
sos = signal.zpk2sos(z, p, k)
gd_w0 = -6*(w0**2-1)/(4*w0**4+17*w0**2+4)
y = signal.sosfilt(sos, x) * 10**(-Hlvl_db_w0/20)
y_env_max_t = np.argmax(np.abs(signal.hilbert(y)))/fs
x_env_max_t = np.argmax(np.abs(signal.hilbert(x)))/fs

plt.subplot(3,1,2)
plt.plot(t,x, 'C0', label=r'input signal $x(t)$')
plt.plot(t,y, 'C1', label=r'output sigal $y(t)$ normalized by system gain')
plt.plot(t,np.abs(signal.hilbert(x)),'C0--', label=r'envelope of $x(t)$')
plt.plot(t,np.abs(signal.hilbert(y)),'C1:', label=r'envelope of $y(t)$')
plt.text(10,0.92, r'analytic result $\tau_\mathrm{GD}$ = %4.3f s' % gd_w0)
plt.text(10,0.82,r'numeric evaluation $\tau_\mathrm{GD}$ = %4.3f s' % (y_env_max_t-x_env_max_t))
plt.xlim(t[0],t[-1])
#plt.xlabel(r'$t$ / s')
plt.ylabel(r'$x(t), y(t)$')
plt.title(r'Minimum Phase System $H(s)_\mathrm{min}=2\,\frac{s+2}{s+1/2}$, $\omega_0=0.1$ rad/s, $\alpha=0.00375$')
plt.legend(loc='upper right')
plt.grid(True)

# Allpass System
sz = +2
sp = -2
H0 = 1
sys = signal.lti(sz, sp, H0)
_, Hlvl_db_w0, _ = signal.bode(sys,  w0)
z, p, k = signal.bilinear_zpk(sz, sp, H0, fs)
_, gdmax_z = signal.group_delay(signal.zpk2tf(z, p, k), w=f, fs=fs)
sos = signal.zpk2sos(z, p, k)
gd_w0 = 4/(w0**2+4)
y = signal.sosfilt(sos, x) * 10**(-Hlvl_db_w0/20)
y_env_max_t = np.argmax(np.abs(signal.hilbert(y)))/fs
x_env_max_t = np.argmax(np.abs(signal.hilbert(x)))/fs

plt.subplot(3,1,3)
plt.plot(t,x, 'C0', label=r'input signal $x(t)$')
plt.plot(t,y, 'C1', label=r'output sigal $y(t)$ normalized by system gain')
plt.plot(t,np.abs(signal.hilbert(x)),'C0--', label=r'envelope of $x(t)$')
plt.plot(t,np.abs(signal.hilbert(y)),'C1:', label=r'envelope of $y(t)$')
plt.text(10,0.92, r'analytic result $\tau_\mathrm{GD}$ = %4.3f s' % gd_w0)
plt.text(10,0.82,r'numeric evaluation $\tau_\mathrm{GD}$ = %4.3f s' % (y_env_max_t-x_env_max_t))
plt.xlim(t[0],t[-1])
plt.xlabel(r'$t$ / s')
plt.ylabel(r'$x(t), y(t)$')
plt.title(r'Allpass $H(s)_\mathrm{all}=\frac{s-2}{s+2}$, $\omega_0=0.1$ rad/s, $\alpha=0.00375$')
plt.legend(loc='upper right')
plt.grid(True)

plt.savefig('envelope_AB91F8317C.pdf')

# Copyright

This tutorial is provided as Open Educational Resource (OER), to be found at
https://github.com/spatialaudio/signals-and-systems-exercises
accompanying the OER lecture
https://github.com/spatialaudio/signals-and-systems-lecture.
Both are licensed under a) the Creative Commons Attribution 4.0 International
License for text and graphics and b) the MIT License for source code.
Please attribute material from the tutorial as *Frank Schultz,
Continuous- and Discrete-Time Signals and Systems - A Tutorial Featuring
Computational Examples, University of Rostock with
``main file, github URL, commit number and/or version tag, year``.